In [1]:
# This notebook illustrates FUNMAN solving PDE models expressed as Petrinets 

# Import funman related code
import os
from funman.api.run import Runner
from funman_demo import summarize_results
from funman_demo.example.pde import plot_spatial_timeseries
from IPython.display import HTML
import matplotlib.pyplot as plt
import itertools


RESOURCES = os.path.join(
   os.getcwd(), "../../Model-Representations/petrinet/examples/pde/"
)


def get_locations(dimensions, num_disc):
   locations = list(itertools.product(*[range(num_disc) for d in range(dimensions)]))
   locations = ["_".join([str(pos) for pos in location]) for location in locations]
   return locations

In [2]:
# Halfar Model

# Instance parameters (used to identify input file)
dimensions = 1
boundary_slope = 0.0
num_disc = 5
# derivative = "centered"
# derivative = "backward"
derivative = "forward"

EXAMPLE_DIR = os.path.join(RESOURCES, "halfar")
instance = f"halfar_{derivative}_{dimensions}_{boundary_slope}_{num_disc}.json"
instance_file = os.path.join(EXAMPLE_DIR, instance)

locations = get_locations(dimensions, num_disc)

# Funman parameters
steps = 10
dt = 10


# Create constraint pos_i on the height h at each location i
# pos_i: 0 <= h_i < 2, for i in [0, 4]

variables = [f"h_{loc}" for loc in locations]
height_bounds = [
    {
        "name": f"pos_{i}",
         "variable": i,
         "interval": {"lb": 0, "ub": 2}
    }
    for i in variables
]

# Create request for the analysis
# structure_parameters: timepoints to evaluate 
# parameters: A, dt
# constraints: height bounds (above), slope on boundaries is well-formed
# config: uses defaults

request_dict = {
    "structure_parameters": [
        {
            "name": "schedules",
            "schedules": [
                {
                    "timepoints": range(0,(steps+1)*dt, dt)
                }
            ],
        },
        
    ],
    "parameters": [
        {
            "name": "A",
            "label":"any",
            "interval": {"lb":1e-20, "ub":1e-12}
         },
        # {
        #     "name": "dt",
        #     "label":"any",
        #     "interval": {"lb":1, "ub":11}
        #  },
         {
            "name": "dx",
            "label":"any",
            "interval": {"lb":1, "ub":1}
         }
    ],
    "constraints": height_bounds + 
    [
        # {"name": "LHS_slope",
        #     "variables": ["h_1", "h_0"],
        #     "weights": [1, -1],
        #     "additive_bounds": {"lb": 0},
        #     "timepoints": {"lb": 0}
        # }, 
        # {"name": "RHS_slope",
        #     "variables": ["h_3", "h_4"],
        #     "weights": [1, -1],
        #     "additive_bounds": {"lb": 0},
        #     "timepoints": {"lb": 0}
        # }
    #  {"name": "melt_h_5",
    #      "variable": "h_5",
    #      "interval": {"lb": 0, "ub": .8},
    #      "timepoints": {"lb": 5}
    #     },

        ],
    "config": {
        "save_smtlib": "./out"
    },
}


# Use request_dict
results = Runner().run(
    instance_file,
    request_dict,
    description=f"Halfar Demo {instance}",
    case_out_dir="./out",
    # dump_plot=True,
    parameters_to_plot=["A", "timestep"],
    point_plot_config = {
        "label_marker":{"true":",", "false": ","},  
    }
)

summarize_results(variables, results)
h, dh, anim_h, anim_dh = plot_spatial_timeseries(results, outdir="./out", fps=dt)
plt.close()
display(h)
plt.close()
display(dh)

[1.00000, 1.00000) has equal lower and upper bounds, so assuming the upper bound is closed.  (I.e., [lb, ub) is actually [lb, ub])
2024-01-29 21:31:43,958 - funman.scenario.scenario - WARNING - Warning: The scenario is not normalized!
2024-01-29 21:32:55,255 - funman.api.run - INFO - Dumping results to ./out/ad6d238f-5793-4c2d-8f2c-edc1e07c7d3a.json
2024-01-29 21:32:55,278 - funman.scenario.consistency - INFO - 10{100}:	[+]
2024-01-29 21:32:55,282 - funman.server.worker - INFO - Completed work on: ad6d238f-5793-4c2d-8f2c-edc1e07c7d3a


In [3]:
# Advection Model

# Instance parameters (used to identify input file)
dimensions = 2
boundary_slope = 0.0
num_disc = 5
# derivative = "centered"
derivative = "backward"
# derivative = "forward"

EXAMPLE_DIR = os.path.join(RESOURCES, "advection")
instance = f"advection_{derivative}_{dimensions}_{boundary_slope}_{num_disc}.json"
instance_file = os.path.join(EXAMPLE_DIR, instance)

locations = get_locations(dimensions, num_disc)

# Funman parameters
steps = 5
dt = 1


# The energy at each location is non-negative
variables = [f"a_{loc}" for loc in locations]
height_bounds = [
    {
        "name": f"pos_{i}",
         "variable": i,
         "interval": {"lb": 0}
    }
    for i in variables
]

request_dict = {
    "structure_parameters": [
        {
            "name": "schedules",
            "schedules": [
                {"timepoints": range(0,(steps+1)*dt, dt)}
            ],
        },
        
    ],
    "parameters":[
        {
            "name": "dx",
            "label":"any",
            #  "interval": {"lb":1e-18, "ub":1e-14}}
            "interval": {"lb":1, "ub":1}},
        {
            "name": "u",
            "label":"any",
            #  "interval": {"lb":1e-18, "ub":1e-14}}
            # "interval": {"lb":-1e-1, "ub":-1e-1} # Best for centered or forward derivatives
            "interval": {"lb":1e-2, "ub":5e-1} # Best for centered or backward derivatives
        }
    ],
    "constraints": 
    height_bounds + 
    [

        # 0 <= dx - u
        # u <= dx
        # {"name": "dx_gte_u",
        #     "variables": ["dx", "u"],
        #     "weights": [1, -1],
        #     "additive_bounds": {"lb": 0},
        #     # "timepoints": {"lb": 0}
        # }, 
 

        ],
    "config": {

    },
}



results = Runner().run(
    instance_file,
    request_dict,
    # REQUEST_PATH,
    description=f"Advection demo {instance}",
    case_out_dir="./out",
    point_plot_config={"variables":variables,},
    num_points=None
)

point_plot = summarize_results(variables, results)
display(point_plot)
plt.close()

h, dh, anim_h, anim_dh = plot_spatial_timeseries(results, outdir="./out")
plt.close()
display(h)
plt.close()
display(dh)

2024-01-29 19:51:16,944 - funman.server.worker - INFO - FunmanWorker running...
[1.00000, 1.00000) has equal lower and upper bounds, so assuming the upper bound is closed.  (I.e., [lb, ub) is actually [lb, ub])
2024-01-29 19:51:16,950 - funman.server.worker - INFO - Starting work on: 302a9155-bf60-42c4-ac43-5d29326f4b99
2024-01-29 19:51:16,953 - funman.scenario.scenario - WARNING - Warning: The scenario is not normalized!
2024-01-29 19:51:18,959 - funman.api.run - INFO - Dumping results to ./out/302a9155-bf60-42c4-ac43-5d29326f4b99.json
2024-01-29 19:51:27,445 - funman.scenario.consistency - INFO - 5{5}:	[+]
2024-01-29 19:51:27,447 - funman.server.worker - INFO - Completed work on: 302a9155-bf60-42c4-ac43-5d29326f4b99
2024-01-29 19:51:29,011 - funman.server.worker - INFO - Worker.stop() acquiring state lock ....
2024-01-29 19:51:29,459 - funman.server.worker - INFO - FunmanWorker exiting...
2024-01-29 19:51:29,460 - funman.server.worker - INFO - Worker.stop() completed.
2024-01-29 19:5

None